In [2]:
import os
import json
import torch
import logging 
import pandas as pd

from collections import defaultdict
from datasets import Dataset
import accelerate
import bitsandbytes

from peft import LoraConfig, LoraModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from trl import SFTTrainer
from quantization import CONFIG_4BITS, CONFIG_4BITS_NESTED, CONFIG_4BITS_NORM, CONFIG_8BITS
from run_utils import *



In [3]:
gemma_train_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Gemma/train.json")
gemma_dev_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Gemma/dev.json")
gemma_test_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Gemma/test.json")

llama_train_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Llama/train.json")
llama_dev_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Llama/dev.json")
llama_test_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Llama/test.json")

mistral_train_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Mistral/train.json")
mistral_dev_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Mistral/dev.json")
mistral_test_dataset = load_tokenized_dataset("/home/andrusha/Desktop/DL Research/Efficient-LLM-Benchmark/UnifiedQA Data Curation/tokenized/Mistral/test.json")

In [4]:
'''
Comment out the one you are not going to be doing. Gemma is already done. This code will load the full model with no quantinzation.
'''

gemma_model, gemma_tokenizer = load_model(base_model="google/gemma-7b", bnb_config=CONFIG_4BITS_NORM, on_gpu=True, use_cache=False, pretraining_tp=1) # Andrey 
# llama2, llama2_tokenizer = load_model("meta-llama/Llama-2-7b-hf") 
# mistral_model, mistral_tokenizer = load_model("mistralai/Mistral-7B-v0.1") 

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
'''
Quantinization Experiments - Loading the model in 4bit and 8bit 
'''

In [ ]:
# code for sending to lambda labs